In [1]:
import numpy as np
import time as tm

In [2]:
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression

In [3]:
def my_fit( X_train, y_train ):
################################
#  Non Editable Region Ending  #
################################
	feat = my_map(X_train)
	model = LogisticRegression(penalty='l2', C=120, fit_intercept=True, max_iter=2500)
	# Use this method to train your model using training CRPs
	# X_train has 32 columns containing the challeenge bits
	# y_train contains the responses
	model.fit(feat, y_train)
	w = model.coef_[0]
	b = model.intercept_
	
	# THE RETURNED MODEL SHOULD BE A SINGLE VECTOR AND A BIAS TERM
	# If you do not wish to use a bias term, set it to 0
	return w, b

In [4]:
def my_map(X):
################################
#  Non Editable Region Ending  #
################################
	X = np.hstack((X, np.ones((X.shape[0], 1))))
	X = 1 - 2 * X
	
	arr = []
	for i in range(X.shape[0]):
		temp = []
		temp.append(X[i][-1])
		for j in range(X.shape[1] - 2, -1, -1):
			temp.append(X[i][j] * temp[-1])
		temp.reverse()
		arr.append(np.array(temp))
    
	arr = np.array(arr)
	
	feat = []
	for i in range(X.shape[0]):
		res = []
		for j in range(X.shape[1]):
			for k in range(j+1, X.shape[1]):
				res.append(arr[i][j] * arr[i][k])
		feat.append(np.array(res))

	feat = np.array(feat)
	feat = np.hstack((feat, np.ones((feat.shape[0], 1))))

	return feat

	# # Use this method to create features.
	# # It is likely that my_fit will internally call my_map to create features for train points

In [8]:
Z_trn = np.loadtxt( "train.dat" )
Z_tst = np.loadtxt( "test.dat" )

n_trials = 1

d_size = 0
t_train = 0
t_map = 0
acc = 0

In [9]:
for t in range( n_trials ):
	tic = tm.perf_counter()
	w, b = my_fit( Z_trn[:, :-1], Z_trn[:,-1] )
	toc = tm.perf_counter()
	t_train += toc - tic

	d_size += w.shape[0]

	tic = tm.perf_counter()
	feat = my_map( Z_tst[:, :-1] )
	toc = tm.perf_counter()
	t_map += toc - tic

	scores = feat.dot( w ) + b
	pred = np.zeros_like( scores )
	pred[scores > 0] = 1
	acc += np.average( Z_tst[ :, -1 ] == pred )

In [10]:
d_size /= n_trials
t_train /= n_trials
t_map /= n_trials
acc /= n_trials

In [11]:
print( d_size, t_train, t_map, acc )

529.0 22.673682630993426 4.0081934010086115 0.9932
